In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
from Genetic_Rehearsal import *

Using TensorFlow backend.


## Load the original data

In [ ]:
import keras
from keras.datasets import mnist
import numpy as np

(x_train_h, y_train_h), (x_test_h, y_test_h) = mnist.load_data()

x_train_h = x_train_h/255
x_test_h = x_test_h/255

# y_train_h = np.copy(y_train_h)
# y_test_h = np.copy(y_test_h)

# x_train_h = np.reshape(x_train_h,(60000,28,28,1))
# x_test_h = np.reshape(x_test_h,(10000,28,28,1))

pre_y_train_h = y_train_h
pre_y_test_h = y_test_h

y_test_h = keras.utils.to_categorical(y_test_h, 20)
y_train_h = keras.utils.to_categorical(y_train_h, 20)

# x_train_h = np.reshape(x_train_h,(60000,784))
# x_test_h = np.reshape(x_test_h,(10000,784))

In [ ]:
import cv2

cv2.imwrite("sample_8.jpg",x_train_h[1045])

True

In [ ]:
# for i in range(20):
print(y_train_h[1045])

8


## Create the model

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, Activation, Dropout, MaxPooling2D

model = Sequential()

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(20))
model.add(Activation('softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
score = model.evaluate(x_test_h,y_test_h,verbose=1)
print(score)

10000/10000 [==============================] - 1s 51us/step
[3.0271900512695313, 0.058400001376867294]


## Train the model on original data

In [ ]:
history_original = model.fit(x_train_h,y_train_h,verbose=1,epochs=3,batch_size=100,validation_data=(x_test_h,y_test_h))

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 6s 101us/step - loss: 0.2529 - accuracy: 0.9252 - val_loss: 0.1155 - val_accuracy: 0.9632
Epoch 2/3
60000/60000 [==============================] - 6s 99us/step - loss: 0.0908 - accuracy: 0.9725 - val_loss: 0.0855 - val_accuracy: 0.9739
Epoch 3/3
60000/60000 [==============================] - 6s 99us/step - loss: 0.0591 - accuracy: 0.9819 - val_loss: 0.0739 - val_accuracy: 0.9775


In [ ]:
model.save("ML_MODEL_2")

# Train the model on Genetic data

Training on genetic data has the following steps

1. Create the genetic data
2. Enrich the generated data
3. Create a model 
4. Train the model on the data

## Creating the genetic data

In [ ]:
help(Genetic_data_Generator)

Help on function Genetic_data_Generator in module Genetic_Rehearsal:

Genetic_data_Generator(model, SHAPE, TARGET_CLASSES, verbose=1, population_size=16, NUMBER_OF_CULTURES=30, NUMBER_OF_GENERATIONS=100, MUTATION_PROBABILITY=0.1, MUTATION_TYPE='+')
    The function takes in the model and target classes and generates synthetic data using Genetic Algorithm.
    OUTPUT FORMAT: [synthetic_data, synthetic_labels]
    
    VARIABLE DESCRIPTION::
    
    model: Input the Keras built neural network.
    SHAPE: Input the shape of sample in dataset. Example for MNIST digits, input ::SHAPE=(28,28,1)
    TARGET_CLASSES: List of all the target classes for which synthetic data is generated.
    verbose: Displays the intermediate output of algorithm if set to 1. Default is 1.
    population_size: Size of population in each culture. Minimum of 4.
    NUMBER_OF_CULTURES: Number of cultures in Genetic algorithm.
    NUMBER_OF_GENERATIONS: Default set to 100. Ignore if selecting other stopping criteria.

In [ ]:
TARGET_CLASSES = [0,1,2,3]
# synthetic_data = Genetic_data_Generator(model,SHAPE=(784,),TARGET_CLASSES = [0,1,2,3,4],verbose=1,population_size=16,NUMBER_OF_CULTURES = 30,NUMBER_OF_GENERATIONS = 100,MUTATION_PROBABILITY = 0.1,MUTATION_TYPE = "+")
synthetic_data = Genetic_data_Generator(model,SHAPE=(28,28),TARGET_CLASSES = [0,1,2,3,4],verbose=1,population_size=16,NUMBER_OF_CULTURES = 30,NUMBER_OF_GENERATIONS = 100,MUTATION_PROBABILITY = 0.1,MUTATION_TYPE = "+")

Class number:  0
Best:  3.7975307554006577  Culture number:  0  Class number:  0
Best:  7.926738262176514  Culture number:  0  Class number:  0
Best:  8.413998037576675  Culture number:  0  Class number:  0
Best:  10.015182942152023  Culture number:  0  Class number:  0
Best:  16.566112637519836  Culture number:  0  Class number:  0
Best:  16.566112637519836  Culture number:  0  Class number:  0
Best:  17.3182874917984  Culture number:  0  Class number:  0
Best:  17.95591413974762  Culture number:  0  Class number:  0
Best:  17.95591413974762  Culture number:  0  Class number:  0
Best:  18.764743208885193  Culture number:  0  Class number:  0
Best:  79.19135689735413  Culture number:  0  Class number:  0
Best:  79.19135689735413  Culture number:  0  Class number:  0
Best:  79.19135689735413  Culture number:  0  Class number:  0
Best:  79.19135689735413  Culture number:  0  Class number:  0
Best:  79.19135689735413  Culture number:  0  Class number:  0
Best:  99.12940859794617  Culture 

KeyboardInterrupt: ignored

In [ ]:
pseudo_x_train_2400 = synthetic_data[0]
pseudo_y_train_2400 = synthetic_data[1]

In [ ]:
# SANITY CHECK

import matplotlib.pyplot as plt

count = [0,0,0,0,0,0,0,0,0,0]
for i in range(pseudo_x_train_2400.shape[0]):
  count[pre_pseudo_y_train_2400[i]]+=1

print(count)
print(sum(count))

plt.bar(range(len(count)),count)
plt.show()

## Enriching the genetic data

### Enrichment step 1

In [ ]:
help(Enrichment)

Help on function Enrichment in module Genetic_Rehearsal:

Enrichment(data, labels, model, NUMBER_OF_CENTERS, NUMBER_OF_CLASSES, NUMBER_OF_SAMPLES, verbose=0)
    Enriches the given data by fitting a Gaussian Mixture model with
    NUMBER_OF_CENTERS and NUMBER_OF_SAMPLES



In [ ]:
NUMBER_OF_SAMPLES = int(pseudo_x_train_2400.shape[0])*2
temp = Enrichment(pseudo_x_train_2400,pseudo_y_train_2400,model,10,20,NUMBER_OF_SAMPLES=NUMBER_OF_SAMPLES)

synthetic_data = temp[0]
synthetic_labels = temp[1]

print("Training data shape: ",synthetic_data.shape)
print("Training labels shape: ",synthetic_labels.shape)

9600/9600 [==============================] - 0s 27us/step
Training data shape:  (9600, 784)
Training labels shape:  (9600, 20)


In [ ]:
# Sanity check:

count = [0,0,0,0,0,0,0,0,0,0]
for i in range(temp[2].shape[0]):
  count[temp[2][i]] += 1

print(count)
print(sum(count))

plt.bar(range(len(count)),count)
plt.show()

### Enrichment Step 2

In [ ]:
NUMBER_OF_SAMPLES = 1000000

temp = Enrichment(synthetic_data,synthetic_labels,model,1,20,NUMBER_OF_SAMPLES)

new_synthetic_data = temp[0]
new_synthetic_labels = temp[1]

print("Training data shape: ",new_synthetic_data.shape)
print("Training labels shape: ",new_synthetic_labels.shape)

1000000/1000000 [==============================] - 23s 23us/step
Training data shape:  (1000000, 784)
Training labels shape:  (1000000, 20)


In [ ]:
pre_new_synthetic = temp[2]

print(pre_new_synthetic.shape)

(1000000,)


In [ ]:
# Sanity check:

count = [0,0,0,0,0,0,0,0,0,0]
for i in range(temp[2].shape[0]):
  count[temp[2][i]] += 1

print(count)
print(sum(count))

plt.bar(range(len(count)),count)
plt.show()

## Train the model on Genetic data

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, Activation, Dropout, MaxPooling2D

model_genetic = Sequential()

model_genetic.add(Dense(512))
model_genetic.add(Activation('relu'))
model_genetic.add(Dropout(0.25))
model_genetic.add(Dense(256))
model_genetic.add(Activation('relu'))
model_genetic.add(Dense(20))
model_genetic.add(Activation('softmax'))
model_genetic.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
# model_genetic.set_weights(initial_weights)

In [ ]:
score = model_genetic.evaluate(x_test_h,y_test_h)
print(score[1])

# history_genetic = model_genetic.fit(pseudo_x_train_2400,pseudo_y_train_2400,verbose=1,epochs=30,batch_size=100,validation_data=(x_test_h,y_test_h),shuffle=True)
# history_genetic = model_genetic.fit(synthetic_data,synthetic_labels,verbose=1,epochs=30,batch_size=100,validation_data=(x_test_h,y_test_h),shuffle=True)
history_genetic = model_genetic.fit(new_synthetic_data,new_synthetic_labels,verbose=1,epochs=30,batch_size=1000,validation_data=(x_test_h,y_test_h),shuffle=True)

10000/10000 [==============================] - 0s 43us/step
0.08330000191926956
Train on 1000000 samples, validate on 10000 samples
Epoch 1/30
1000000/1000000 [==============================] - 7s 7us/step - loss: 1.0261 - accuracy: 0.6116 - val_loss: 2.0703 - val_accuracy: 0.5463
Epoch 2/30
1000000/1000000 [==============================] - 6s 6us/step - loss: 0.7509 - accuracy: 0.7065 - val_loss: 2.5148 - val_accuracy: 0.5635
Epoch 3/30
1000000/1000000 [==============================] - 7s 7us/step - loss: 0.6710 - accuracy: 0.7363 - val_loss: 2.3938 - val_accuracy: 0.6042
Epoch 4/30
1000000/1000000 [==============================] - 7s 7us/step - loss: 0.6249 - accuracy: 0.7539 - val_loss: 2.4549 - val_accuracy: 0.6143
Epoch 5/30
1000000/1000000 [==============================] - 6s 6us/step - loss: 0.5920 - accuracy: 0.7653 - val_loss: 2.6143 - val_accuracy: 0.6234
Epoch 6/30
1000000/1000000 [==============================] - 6s 6us/step - loss: 0.5691 - accuracy: 0.7747 - val_loss

In [ ]:
del new_synthetic_data
del new_synthetic_labels